In [2]:
import pandas as pd
import numpy as np
import sys
from prophet import Prophet

c:\Users\AlejandroLuzanto\.conda\envs\prophet\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sys.path.append('../scr')

from utils_clean import filtrar_ub_mensual, partir_train_test
from utils_data import downloadTable
from utils_metrics import errorMetrics

In [19]:
queryPrices = """
SELECT * FROM `desarrollo-444913.globalPrices.prices`
"""
# Usar forceDownload=True para actualizar la tabla
df = downloadTable(queryPrices,"globalPricesTable",forceDownload=False)
dfUbMensual = filtrar_ub_mensual(df)
train = dfUbMensual.loc[:'2023-12-01']   # Ajusta el string según tu índice
# Test: enero, febrero, marzo 2025
test = dfUbMensual.loc['2024-01-01':'2024-12-01']



Folder 'bigqueryDatabases' already exists.
Reading bigqueryDatabases\globalPricesTable.csv from local CSV.


In [ ]:
train = pd.DataFrame(train, columns=['DS', 'Y'])
train['DS'] = pd.to_datetime(train['DS'])
train.set_index('DS', inplace=True)
#train = train.resample('MS').mean().reset_index()

In [21]:
train

,Y
DS,


# Prophet horizonte de 12 meses

year_month
2007-01-01    4.173396
2007-02-01    4.213685
2007-03-01    4.106710
2007-04-01    4.207433
2007-05-01    4.111156
                ...   
2025-02-01    6.270513
2025-03-01    6.308024
2025-04-01    6.435143
2025-05-01    6.079487
2025-06-01    5.551561
Name: price, Length: 222, dtype: float64

# Prophet one step refitting model